In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("fmnist_small.csv")

# Reshape images into sequences

In [3]:
# — convert each image (28×28) into 28 time steps of 28 features each
x = df.iloc[:, 1:].values.reshape(-1, 28, 28)
y = df.iloc[:, 0].values

# Normalize the data
x = x / 255.0

# Split into train/val/test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

# Dataset and data loader

In [5]:
# Dataset class
class dataset(Dataset):

    def __init__ (self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
train_dataset = dataset(X_train, y_train)
test_dataset = dataset(X_test, y_test)

In [6]:
# — Vanilla LSTM
class lstm_model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(lstm_model, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
model = lstm_model(input_size = 28, hidden_size=128, num_layers = 2, num_classes=10)

In [ ]:
# — CrossEntropyLoss + Adam
cri